## Azure 인증 설정

In [1]:
import os
from azure.identity import AzureCliCredential

# ============================================
# 🔐 인증 방식 선택 (둘 중 하나만 True로 설정!)
# ============================================
USE_API_KEY = True           # API Key 기반 인증
USE_CLI_CREDENTIAL = False   # AzureCliCredential 사용 (az login 필요)

# ============================================

if USE_API_KEY and USE_CLI_CREDENTIAL:
    raise ValueError("❌ 둘 중 하나만 True로 설정하세요!")

if not USE_API_KEY and not USE_CLI_CREDENTIAL:
    raise ValueError("❌ 최소 하나는 True로 설정하세요!")

if USE_API_KEY:
    # API Key 인증 - 아래 값을 본인 정보로 수정!
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY", "").strip()  # 👈 API Key 입력
    
    credential = None  # API Key 사용 시 credential 불필요
    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    
    print("🔐 API Key 인증 사용")
    print("   (API Key로 Azure OpenAI에 직접 연결)")
    
    if AZURE_OPENAI_API_KEY == "your-api-key-here":
        print("\n⚠️ API Key를 실제 값으로 수정하세요!")
    else:
        print("\n✅ API Key 설정 완료!")

elif USE_CLI_CREDENTIAL:
    credential = AzureCliCredential()
    print("🔐 AzureCliCredential 사용")
    print("   (터미널에서 'az login' 필요)")
    
    # 인증 테스트
    try:
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print("\n✅ Azure CLI 인증 성공!")
    except Exception as e:
        print(f"\n❌ 인증 실패: {e}")
        print("   터미널에서 'az login'을 실행하세요.")

🔐 API Key 인증 사용
   (API Key로 Azure OpenAI에 직접 연결)

✅ API Key 설정 완료!


## Azure OpenAI 설정

In [2]:
import os

#  아래 값을 본인의 Azure OpenAI 정보로 수정하세요!
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://hbjoo-mlg6s9pj-eastus2.cognitiveservices.azure.com"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o"  # 또는 gpt-4.1

print("✅ Azure OpenAI 설정 완료!")
print(f"   Endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}")
print(f"   Deployment: {os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME']}")

✅ Azure OpenAI 설정 완료!
   Endpoint: https://hbjoo-mlg6s9pj-eastus2.cognitiveservices.azure.com
   Deployment: gpt-4o


## STEP 1️⃣ DSR 기반 최대 대출 계산

In [3]:
# 시중 은행 주택담보대출 평균 금리 조회 (한국은행 ECOS API)
# DATA/mortgage_rate.csv에 해당 월 금리가 있으면 사용, 없을 때만 API 호출 후 CSV에 저장
# 환경변수 BOK_ECOS_API_KEY 설정 시 API 사용, 미설정 시 기본값 4.48% 사용
import os
import csv
from datetime import date, datetime
from pathlib import Path
from dotenv import load_dotenv

# .env를 cwd부터 상위 폴더로 찾아서 로드 (노트북 cwd가 프로젝트 루트가 아닐 수 있음)
_cwd = Path.cwd().resolve()
for _ in range(6):
    _env = _cwd / ".env"
    if _env.exists():
        load_dotenv(_env)
        break
    if _cwd.parent == _cwd:
        break
    _cwd = _cwd.parent
else:
    load_dotenv()
print("BOK_ECOS_API_KEY:", "설정됨" if os.environ.get("BOK_ECOS_API_KEY", "").strip() else "미설정 (프로젝트 루트 .env 확인)")

_DEFAULT_RATE = 0.0448  # API 미사용 시 적용 (연 4.48%)

def _csv_path():
    return Path("DATA") / "mortgage_rate.csv"

def _minus_months(d, months):
    """날짜에서 months개월 뺀 날짜 반환 (월 단위)"""
    y, m = d.year, d.month
    m -= months
    while m <= 0:
        m += 12
        y -= 1
    return date(y, m, 1)

def _save_rate_to_csv(csv_path, yyyymm, rate):
    """해당 월 금리를 CSV에 한 줄 추가"""
    try:
        csv_path = Path(csv_path).resolve()
        csv_path.parent.mkdir(parents=True, exist_ok=True)
        file_exists = csv_path.exists()
        need_header = not file_exists or csv_path.stat().st_size == 0
        with open(csv_path, "a", encoding="utf-8", newline="") as f:
            w = csv.DictWriter(f, fieldnames=["yyyymm", "rate"])
            if need_header:
                w.writeheader()
            w.writerow({"yyyymm": yyyymm, "rate": rate})
    except Exception:
        pass

def get_mortgage_rate(as_of_date=None):
    """
    해당 날짜 기준 예금은행 주택담보대출 금리(연%)를 소수로 반환.
    ECOS는 약 2달 지연 공표이므로, 요청일 기준 2달 전 월 데이터를 API로 조회함.
    """
    if as_of_date is None:
        as_of_date = date.today()
    if isinstance(as_of_date, datetime):
        as_of_date = as_of_date.date()
    yyyymm = as_of_date.strftime("%Y%m")
    effective_date = _minus_months(as_of_date, 2)
    effective_yyyymm = effective_date.strftime("%Y%m")
    csv_path = _csv_path().resolve()
    if csv_path.exists():
        try:
            with open(csv_path, "r", encoding="utf-8") as f:
                r = csv.DictReader(f)
                for row in r:
                    if row.get("yyyymm") == yyyymm and row.get("rate", "").strip():
                        return float(row["rate"].strip())
        except Exception:
            pass
    api_key = os.environ.get("BOK_ECOS_API_KEY", "").strip()
    rate = None
    if api_key:
        try:
            import urllib.request
            import json
            # 121Y006 + BECBLA0302: 예금은행 대출금리(신규취급액 기준) 주택담보대출, 현재일 기준 2달 전 월 조회
            url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/10/121Y006/M/{effective_yyyymm}/{effective_yyyymm}/BECBLA0302/"
            print("url: ",url)
            with urllib.request.urlopen(url, timeout=10) as res:
                data = json.loads(res.read().decode())
            stat = data.get("StatisticSearch", {})
            result = stat.get("RESULT", {})
            if isinstance(result, dict) and result.get("CODE") != "000":
                pass
            else:
                rows = stat.get("row") or []
                if isinstance(rows, dict):
                    rows = [rows]
                if rows:
                    val = rows[0].get("DATA_VALUE")
                    if val is not None and str(val).strip():
                        rate = float(str(val).replace(",", ".").strip()) / 100.0
        except Exception:
            pass
        if rate is None:
            print(f"[get_mortgage_rate] ECOS API 호출했으나 금리 추출 실패. 기본금리({_DEFAULT_RATE*100:.2f}%) 사용.")
    if rate is None:
        rate = _DEFAULT_RATE
    _save_rate_to_csv(csv_path, yyyymm, rate)
    return rate


# DSR = (연간 모든 대출 원리금 상환액) / 연소득
def loan_by_dsr(annual_income, is_first_home=False, as_of_date=None):
    """
    is_first_home: 생애최초 구매 여부 (True면 DSR 0.7, 아니면 0.4)
    as_of_date: 금리 적용 기준일 (date/datetime). None이면 오늘 기준 시중 금리 사용.
    """
    dsr = 0.7 if is_first_home else 0.4
    max_annual = annual_income * dsr
    r = get_mortgage_rate(as_of_date)
    n = 30
    factor = ((1 + r) ** n - 1) / (r * (1 + r) ** n)
    return max_annual * factor

BOK_ECOS_API_KEY: 설정됨


## STEP 2️⃣ 취득세 함수

In [4]:
def acquisition_tax(price):
    if price <= 600_000_000:
        return price * 0.011   # 1.1%
    elif price <= 900_000_000:
        return price * 0.022   # 근사치
    else:
        return price * 0.033   # 3.3%

## STEP 3️⃣ 전체 최대 매수가격 계산 (반복 탐색)

In [5]:
## 최대 금액 계산기 로직
def max_purchase_price(
    annual_income,
    ltv,
    cash,
    is_first_home=False,
    as_of_date=None
):
    """as_of_date: 금리 적용 기준일. None이면 오늘 기준 시중 주택담보대출 평균 금리."""
    loan_dsr_limit = loan_by_dsr(annual_income, is_first_home=is_first_home, as_of_date=as_of_date)
    
    price = 100_000_000  # 시작값 1억
    step = 10_000_000    # 1천만원 단위 탐색
    
    max_price = 100_000_000
    
    while price < 200_000_000_000:
        loan_ltv = price * ltv
        loan = min(loan_dsr_limit, loan_ltv)
        
        tax = acquisition_tax(price)
        
        if price + tax <= loan + cash:
            max_price = price
            price += step
        else:
            break

    print("대출 가능 금액: ", loan)
    
    return max_price

## 호출 테스트

In [6]:
# as_of_date 생략 시 오늘 기준 시중 주택담보대출 평균 금리 사용 (BOK ECOS API 또는 기본 4.48%)
result = max_purchase_price(
    annual_income=160_000_000,
    ltv=0.7,
    cash=320_000_000,
    is_first_home=True,
    as_of_date=None  # 예: date(2024, 6, 1) 지정 시 해당 월 금리 적용
)
print(f"최대 매수 가능 금액: {result:,} 원")

대출 가능 금액:  679000000.0
최대 매수 가능 금액: 960,000,000 원


## 메모리 클래스 정의

In [18]:
from agent_framework import ContextProvider, Context, ChatMessage
from collections.abc import Sequence, MutableSequence
from typing import Any
import re

class LoanInfoMemory(ContextProvider):
    """사용자 연소득, 보유현금, 첫구매 여부 정보를 기억하는 메모리"""
    
    def __init__(self, chat_client):
        self._chat_client = chat_client
        self.loan_info = {
            "annual_income": None,  # 연소득 (원)
            "cash": None,  # 보유현금 (원)
            "is_first_home": None  # 생애최초구매여부 (True/False)
        }
    
    def _extract_amount(self, text: str, keywords: list) -> float | None:
        """텍스트에서 키워드 주변 금액 추출 (억, 천만, 만, 원 단위 지원)"""
        text_lower = text.lower()
        
        def _to_krw(num_str: str, unit: str) -> int:
            num = float(num_str)
            if unit in ['억', '억원']:
                return int(num * 100_000_000)
            if unit in ['천만', '천만원']:
                return int(num * 10_000_000)
            if unit in ['만', '만원']:
                return int(num * 10_000)
            return int(num)
        
        for keyword in keywords:
            if keyword not in text_lower:
                continue
            
            # 1) 키워드 뒤 금액 우선 (예: "현금은 4억")
            pattern_after = rf'{keyword}\D*(\d+(?:\.\d+)?)\s*(억|천만|만|원|만원|억원)?'
            match_after = re.search(pattern_after, text)
            if match_after:
                return _to_krw(match_after.group(1), match_after.group(2) or '')
            
            # 2) 키워드 앞 금액 (예: "4억 현금")
            pattern_before = rf'(\d+(?:\.\d+)?)\s*(억|천만|만|원|만원|억원)?\D*{keyword}'
            match_before = re.search(pattern_before, text)
            if match_before:
                return _to_krw(match_before.group(1), match_before.group(2) or '')
        
        return None
    
    def _extract_info_from_text(self, text: str) -> None:
        """텍스트에서 연소득, 보유현금, 첫구매 여부 추출"""
        text_lower = text.lower()
        
        # 연소득 추출 (새 값이 있으면 갱신)
        income = self._extract_amount(text, ["연소득", "연봉", "연수입", "연수익", "연간소득"])
        if income is not None:
            self.loan_info["annual_income"] = income
        
        # 보유현금 추출 (새 값이 있으면 갱신)
        cash = self._extract_amount(text, ["보유현금", "현금", "예금", "저축", "보유자금", "자금", "보유"])
        if cash is not None:
            self.loan_info["cash"] = cash
        
        # 생애최초구매 여부 추출 (새 표현이 들어오면 기존 값 갱신)
        negated_first_home = re.search(r'(생애최초|첫\s?구매|첫\s?주택|처음\s?사는).{0,8}(아니|아님|아니요|아니오|아니에요)', text_lower)
        if negated_first_home:
            self.loan_info["is_first_home"] = False
        elif any(k in text_lower for k in ["생애최초", "처음", "첫구매", "첫 주택", "처음 사는"]):
            self.loan_info["is_first_home"] = True
        elif any(k in text_lower for k in ["두번째", "2주택", "재구매", "이미", "생애최초 아님", "첫구매 아님"]):
            self.loan_info["is_first_home"] = False
    
    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage] = None, **kwargs) -> Context | None:
        """에이전트 호출 전: 저장된 정보를 컨텍스트로 제공"""
        info_parts = []
        if self.loan_info["annual_income"]:
            info_parts.append(f"- 연소득: {self.loan_info['annual_income']:,}원")
        if self.loan_info["cash"]:
            info_parts.append(f"- 보유현금: {self.loan_info['cash']:,}원")
        if self.loan_info["is_first_home"] is not None:
            info_parts.append(f"- 생애최초구매: {'예' if self.loan_info['is_first_home'] else '아니오'}")
        
        if info_parts:
            info = "[사용자 정보]\n" + "\n".join(info_parts) + "\n\n위 정보를 참고하여 대출 계산 및 추천을 제공하세요."
            return Context(instructions=info)
        return None
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """에이전트 호출 후: 사용자 메시지에서 정보 추출"""
        if isinstance(request_messages, ChatMessage):
            messages_list = [request_messages]
        else:
            messages_list = list(request_messages)
        
        for msg in messages_list:
            if hasattr(msg, 'role') and msg.role.value == "user":
                text = ""
                if hasattr(msg, 'contents') and msg.contents:
                    for content in msg.contents:
                        if hasattr(content, 'text'):
                            text += content.text
                        elif isinstance(content, str):
                            text += content
                        elif isinstance(content, dict) and 'text' in content:
                            text += content['text']
                
                if text:
                    self._extract_info_from_text(text)

print("✅ LoanInfoMemory 정의 완료")

✅ LoanInfoMemory 정의 완료


## 에이전트 생성

In [22]:
from agent_framework.azure import AzureOpenAIChatClient

# Budget Agent 생성
if USE_API_KEY:
    chat_client = AzureOpenAIChatClient(
        api_key=os.environ["AZURE_OPENAI_API_KEY"]
    )
else:
    chat_client = AzureOpenAIChatClient(credential=credential)

# 메모리 생성 및 Agent에 연결
loan_memory = LoanInfoMemory(chat_client)

agent = chat_client.as_agent(
    instructions="""
    아파트를 구매하는 사람의 최대 대출 가능 금액을 계산하는 어시스턴트입니다.
    사용자에게 아래의 내용을 물어보고 메모리에 저장하세요.
    - 연소득(부부라면 합산 연소득)
    - 생애첫구매여부
    - 보유현금
    
    항상 친절하고 간결하게, 한국어로 응답합니다.
    """,
    name="BudgetAgent",
    context_provider=loan_memory
)
# Thread 생성 (메모리와 함께 사용)
thread = agent.get_new_thread()

print("✅ Budget Agent 생성 완료!")

✅ Budget Agent 생성 완료!


## 메모리 생성 및 Agent 연결

In [21]:
print("✅ LoanInfoMemory 생성 및 Agent 연결 완료!")
print(f"   초기 상태: 연소득={loan_memory.loan_info['annual_income']}, 보유현금={loan_memory.loan_info['cash']}")

✅ LoanInfoMemory 생성 및 Agent 연결 완료!
   초기 상태: 연소득=None, 보유현금=None


## Agent 사용 예시

In [20]:
# 사용자에게 연소득과 보유현금을 물어보는 메시지
agent_result = await agent.run("안녕하세요! 아파트 구매를 고민 중이예요", thread=thread)

print("🤖 Agent 응답:")
print(agent_result.text)

# 메모리에 저장된 정보 확인
print("\n📝 메모리에 저장된 정보:")
print(f"   연소득: {loan_memory.loan_info['annual_income'] or '아직 입력 안됨'}")
print(f"   보유현금: {loan_memory.loan_info['cash'] or '아직 입력 안됨'}")
print(f"   생애최초구매: {loan_memory.loan_info['is_first_home'] if loan_memory.loan_info['is_first_home'] is not None else '아직 입력 안됨'}")

🤖 Agent 응답:
안녕하세요! 아파트 구매를 고려 중이시군요. 최대 대출 가능 금액을 계산하는 데 필요한 몇 가지 정보를 여쭤봐도 될까요?

1. 연소득(부부라면 합산 연소득)은 얼마인가요?  
2. 생애 첫 구매인가요?  
3. 현재 보유하고 계신 현금은 얼마나 되시나요?  

📝 메모리에 저장된 정보:
   연소득: 아직 입력 안됨
   보유현금: 아직 입력 안됨
   생애최초구매: 아직 입력 안됨


## 에이전트 테스트

In [23]:
r1 = await agent.run("연봉은 남편이랑 저 합쳐서 3억 정도고요, 현금은 1억 갖고 있어요. ", thread=thread)
print("🤖 Turn 1:", r1, "...\n")

🤖 Turn 1: 알겠습니다! 생애 첫 구매인지 말씀해주세요. 생애 첫 구매라면 최대 대출 가능 금액이 달라질 수 있습니다. ...



In [24]:
r1 = await agent.run("우리 둘 다 생애최초가 아니예요", thread=thread)
print("🤖 Turn 1:", r1, "...\n")

🤖 Turn 1: 감사합니다! 아래 정보를 기반으로 대략적인 대출 가능 금액을 계산해 보겠습니다.

- 연소득: 3억 원
- 생애최초 구매 아님
- 보유 현금: 1억 원

보통 연소득의 5~7배까지 대출이 가능하며, 주택담보대출비율(LTV)에 따라 실제 대출 금액이 조정됩니다. 생애 첫 구매가 아니므로 일반적인 경우를 적용하겠습니다.

대출 금액 추정:
- 연소득 기준: 약 15억~21억 원
- LTV 기준 (약 60~70%): 구매하려는 아파트 가격에 따라 달라짐

따라서 구매하려는 아파트 가격을 말씀해 주시면 더 정확한 계산을 도와드릴 수 있습니다! 😊 ...



In [17]:
# 메모리에 저장된 정보 확인
print("\n📝 메모리에 저장된 정보:")
print(f"   연소득: {loan_memory.loan_info['annual_income'] or '아직 입력 안됨'}")
print(f"   보유현금: {loan_memory.loan_info['cash'] or '아직 입력 안됨'}")
print(f"   생애최초구매: {loan_memory.loan_info['is_first_home'] if loan_memory.loan_info['is_first_home'] is not None else '아직 입력 안됨'}")


📝 메모리에 저장된 정보:
   연소득: 300000000
   보유현금: 100000000
   생애최초구매: True
